# Compressed Matrix Factorization

In [1]:
import numpy as np
import scipy.sparse as sparse
import matrix_factorization as mf
import metrics
from collections import defaultdict

In [2]:
def _get_random_state(random_state):
    if type(random_state) == np.random.RandomState:
        random = random_state
    else:
        random = np.random.RandomState(random_state)
    return random

## Projection / measurement matrices

We consider sparse binary projection matrices $P$ that correspond to the adjacency matrix of a bipartite graph where each node on the left-side of the graph has exactly $s$ neighbors. We use a simple randomized construction: for each column of the adjacency matrix, set $s$ distinct entries chosen uniformly at random to $1$. Informally speaking, random graphs of this form are known to be *bipartite expanders* with high probability -- a property that we use in the theoretical analysis in the paper.

In [3]:
def sample_projection(d, n, s, dtype=np.float32, random_state=None):
    """Sample a sparse binary projection matrix.
    
    Args:
        d: int, number of rows/projection dimension
        n: int, number of columns/input dimension
        s: int, column sparsity of matrix
        dtype: data type
        random_seed: random seed
        
    Returns:
        Projection matrix of type scipy.sparse.csr.csr_matrix
    """
    random = _get_random_state(random_state)
    adj = np.zeros([n, s], dtype=np.int32)
    for i in range(n):
        adj[i] = random.choice(d, replace=False, size=s)
    P = sparse.lil_matrix((d, n), dtype=dtype)
    for i in range(n):
        for j in random.choice(d, replace=False, size=s):
            P[j, i] = 1
    return P.tocsr()

## Synthetic data

In [4]:
def generate_synthetic(W_sparsity=100, rank=10, n=2000, m=2000, d=400,
                       P_sparsity=5, nonneg=False, noise_scale=0.1, 
                       random_state=None, dtype=np.float32, verbose=False):
    """Generate a synthetic low-rank + noise matrix.
    
    Args:
        W_sparsity: int, column sparsity of the left factor W
        rank: int, rank parameter
        n: int, number of rows of the uncompressed matrix
        m: int, number of columns
        d: int, number of rows of the compressed matrix
        nonneg: bool, if true, then W and H are nonnegative
        noise_scale: float, Frobenius norm of noise matrix as fraction of norm of the noiseless matrix
        random_seed: random seed
        dtype: str or dtype, datatype of generated data
        verbose: bool, whether to print parameters
        
    Returns:
        dict containing generated data
    """
    
    if verbose:
        print('W_sparsity =', W_sparsity)
        print('rank =', rank)
        print('n =', n)
        print('m =', m)
        print('d =', d)
        print('P_sparsity =', P_sparsity)
        print('nonneg =', nonneg)
        print('noise_scale =', noise_scale)
    
    random = _get_random_state(random_state)
    W = random.randn(n, rank).astype(dtype)
    for i in range(rank):
        s = np.zeros(n, dtype=dtype)
        s[random.choice(n, size=W_sparsity, replace=False)] = 1
        W[:, i] *= s
    
    H = random.randn(rank, m).astype(dtype)
    for i in range(rank):
        H[i] /= np.linalg.norm(H[i])
    
    if nonneg:
        W = np.abs(W)
        H = np.abs(H)
        
    X_true = W.dot(H)
    
    noise = random.randn(n, m).astype(dtype)
    noise *= noise_scale * np.linalg.norm(X_true, 'fro') / np.linalg.norm(noise, 'fro')
    X = X_true + noise
    
    P = sample_projection(d, n, s=P_sparsity, random_state=random, dtype=dtype)
    return dict(X=X, X_true=X_true, W=W, H=H, P=P, PX=P.dot(X), PW=P.dot(W))

## Experiments

In [5]:
def synthetic_nmf_experiment(rank=10, nonneg_recovery=False, random_state=None, verbose=False, show_progress=False, **kwargs):
    random = _get_random_state(random_state)
    
    # generate synthetic low-rank + noise matrix
    data = generate_synthetic(rank=rank, nonneg=True, random_state=random, verbose=verbose, **kwargs)
    
    # factorize and recover
    factors = mf.compressed_factorization(
        data['PX'], 
        rank=rank, 
        fn=mf.nmf, 
        proj=data['P'], 
        nonneg=nonneg_recovery,
        random_state=random,
        show_progress=show_progress,
        verbose=verbose)
        
    res = defaultdict(dict)
    
    # factor matching
    What, Hhat = metrics.normalize_factors(factors['W'], factors['H'])
    Wtilde = metrics.normalize_factors(factors['PW'], factors['H'])[0]
    res['W']['fro'] = metrics.match_l2(data['W'], What, match_rows=False)['score']
    res['W']['corr'] = metrics.match_correlation(data['W'], What, match_rows=False)['score']
    res['H']['fro'] = metrics.match_l2(data['H'], Hhat, match_rows=True)['score']
    res['H']['corr'] = metrics.match_correlation(data['H'], Hhat, match_rows=True)['score']
    res['PW']['fro'] = metrics.match_l2(data['PW'], Wtilde, match_rows=False)['score']
    res['PW']['corr'] = metrics.match_correlation(data['PW'], Wtilde, match_rows=False)['score']
    
    # reconstruction error
    res['X']['uncompressed'] = metrics.error(
        data['X'], np.dot(*mf.nmf(data['X'], rank, random_state=random)))
    res['X']['compressed'] = metrics.error(data['X'], np.dot(What, Hhat))
    
    return res

In [6]:
synthetic_nmf_experiment(d=400, verbose=True, show_progress=True)

W_sparsity = 100
rank = 10
n = 2000
m = 2000
d = 400
P_sparsity = 5
nonneg = True
noise_scale = 0.1


 34%|███▍      | 342/1000 [00:00<00:00, 1704.18it/s]

Computing rank 10 NMF of matrix of shape (400, 2000)


 20%|██        | 2/10 [00:00<00:00, 10.58it/s]

Solving 10 sparse recovery instances of dimension 2000


100%|██████████| 10/10 [00:01<00:00,  8.86it/s]


defaultdict(dict,
            {'W': {'fro': 0.049148860902445593, 'corr': 0.9991041959605538},
             'H': {'fro': 0.022943871061640028, 'corr': 0.9994026481422557},
             'PW': {'fro': 0.022912487975926033, 'corr': 0.9997903942427104},
             'X': {'uncompressed': 0.099175327, 'compressed': 0.10979814}})

In [7]:
def synthetic_spca_experiment(alpha=0.1, rank=10, random_state=None, verbose=False, show_progress=False, **kwargs):
    random = _get_random_state(random_state)
    
    # generate synthetic low-rank + noise matrix
    data = generate_synthetic(rank=rank, nonneg=False, random_state=random, verbose=verbose, **kwargs)
    
    # factorize and recover
    factors = mf.compressed_factorization(
        data['PX'],
        rank=rank,
        fn=mf.sparse_pca,
        alpha=alpha,
        proj=data['P'],
        random_state=random,
        show_progress=show_progress,
        verbose=verbose)
    
    res = defaultdict(dict)
    
    # factor matching
    What, Hhat = metrics.normalize_factors(factors['W'], factors['H'])
    Wtilde = metrics.normalize_factors(factors['PW'], factors['H'])[0]
    res['W']['fro'] = metrics.match_l2(data['W'], What, match_rows=False)['score']
    res['W']['corr'] = metrics.match_correlation(data['W'], What, match_rows=False)['score']
    res['H']['fro'] = metrics.match_l2(data['H'], Hhat, match_rows=True)['score']
    res['H']['corr'] = metrics.match_correlation(data['H'], Hhat, match_rows=True)['score']
    res['PW']['fro'] = metrics.match_l2(data['PW'], Wtilde, match_rows=False)['score']
    res['PW']['corr'] = metrics.match_correlation(data['PW'], Wtilde, match_rows=False)['score']
    
    # reconstruction error
    res['X']['uncompressed'] = metrics.error(
        data['X'], np.dot(*mf.sparse_pca(data['X'], rank, alpha, random_state=random)))
    res['X']['compressed'] = metrics.error(data['X'], np.dot(What, Hhat))
    
    return res

In [8]:
synthetic_spca_experiment(d=400, alpha=0.1, verbose=True, show_progress=True)

W_sparsity = 100
rank = 10
n = 2000
m = 2000
d = 400
P_sparsity = 5
nonneg = False
noise_scale = 0.1
Computing rank 10 sparse PCA of matrix of shape (400, 2000) with L1 parameter 0.1


 10%|█         | 1/10 [00:00<00:01,  6.27it/s]

Solving 10 sparse recovery instances of dimension 2000


100%|██████████| 10/10 [00:01<00:00,  6.04it/s]


defaultdict(dict,
            {'W': {'fro': 0.26148362665759389, 'corr': 0.9741499153332656},
             'H': {'fro': 0.041768433590831398, 'corr': 0.9991275702322824},
             'PW': {'fro': 0.087209960319630186, 'corr': 0.9978464818690478},
             'X': {'uncompressed': 0.13998630533221196,
              'compressed': 0.27979464042601676}})